In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import requests as req
import sys
import time
sys.path.insert(0, '../../../API_keys')
from vantage_api import alpha_vantage_key
import json

In [2]:
proxies = {'https': 'http://127.0.0.1:7769'} #I need this on my computer

In [3]:
df_senate = pd.read_csv('loren_data/loren_complete_data.csv')

In [4]:
df_senate.head()

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
1,2022-05-31,X,"United States Steel Corporation <div class=""te...",sale_partial,50000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
2,2022-05-31,X,"United States Steel Corporation <div class=""te...",sale_full,50000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
3,2022-05-31,X,United States Steel Corporation,purchase,250000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
4,2022-05-20,PYPL,"PayPal Holdings, Inc. -",purchase,100000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL


In [5]:
df_house = pd.read_csv('loren_data/clean_house_2022-07-15.csv')

In [6]:
df_house.head()

,disclosure_date,transaction_date,ticker,asset_description,type,amount,name,represents,chamber,party,birthday,gender
0,2021-10-04,2021-09-27,BP,BP plc,purchase,15000,Virginia Foxx,NC05,house,Republican,1943-06-29,F
1,2021-10-04,2021-09-13,XOM,Exxon Mobil Corporation,purchase,15000,Virginia Foxx,NC05,house,Republican,1943-06-29,F
2,2021-10-04,2021-09-10,ILPT,Industrial Logistics Properties Trust - Common...,purchase,50000,Virginia Foxx,NC05,house,Republican,1943-06-29,F
3,2021-10-04,2021-09-28,PM,Phillip Morris International Inc,purchase,50000,Virginia Foxx,NC05,house,Republican,1943-06-29,F
4,2021-10-04,2021-09-17,BLK,BlackRock Inc,sale_partial,15000,Alan S. Lowenthal,CA47,house,Democrat,1941-03-08,M


In [7]:
# combining the two dataframes
congress = pd.concat([df_senate, df_house], ignore_index=True)
  
# display combined dataframes
congress

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
1,2022-05-31,X,"United States Steel Corporation <div class=""te...",sale_partial,50000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
2,2022-05-31,X,"United States Steel Corporation <div class=""te...",sale_full,50000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
3,2022-05-31,X,United States Steel Corporation,purchase,250000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
4,2022-05-20,PYPL,"PayPal Holdings, Inc. -",purchase,100000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34649,2020-04-09,SWK,"Stanley Black & Decker, Inc.",sale_partial,15000,Ed Perlmutter,2020-06-10,house,NaN,NaN,Democrat,1953-05-01,M,CO07
34650,2020-04-09,USB,U.S. Bancorp,sale_partial,15000,Ed Perlmutter,2020-06-10,house,NaN,NaN,Democrat,1953-05-01,M,CO07
34651,2020-03-13,BMY,Bristol-Myers Squibb Company,sale_full,250000,Nicholas Van Taylor,2020-06-10,house,NaN,NaN,Republican,1972-08-01,M,TX03
34652,2020-03-13,LLY,Eli Lilly and Company,sale_full,1000000,Nicholas Van Taylor,2020-06-10,house,NaN,NaN,Republican,1972-08-01,M,TX03


In [8]:
ticker1 = pd.read_csv('loren_data/updated_ticker_symbols-v1.csv')
ticker1.head()

,symbol,name
0,AAPL,Apple Inc.
1,XOM,Exxon Mobil Corporation
2,MSFT,Microsoft Corporation
3,BAC^I,Bank of America Corporation
4,IBM,International Business Machines Corporation


In [9]:
ticker2 = pd.read_csv('loren_data/updated_ticker_symbols-v2.csv')
ticker2.head()

,symbol,name
0,AAPL,Apple Inc.
1,XOM,Exxon Mobil Corporation
2,MSFT,Microsoft Corporation
3,BAC^I,Bank of America Corporation
4,IBM,International Business Machines Corporation


In [10]:
tick_list1 = [n for n in ticker1['symbol'].unique()]
tick_list2 = [n for n in ticker2['symbol'].unique()]
not_shared = []
for i in tick_list1:
    if i not in tick_list2:
        not_shared.append(i)
not_shared

[]

Tickers are the same

In [11]:
tick_list1

['AAPL',
 'XOM',
 'MSFT',
 'BAC^I',
 'IBM',
 'CVX',
 'GE',
 'WMT',
 'T',
 'JNJ',
 'PG',
 'WFC',
 'KO',
 'PFE',
 'JPM',
 'GOOG',
 'PM',
 'VOD',
 'ORCL',
 'INTC',
 'MRK',
 'VZ',
 'BRK/A',
 'QCOM',
 'PEP',
 'CSCO',
 'AMZN',
 'ABT',
 'MCD',
 'SLB',
 'C',
 'BRK/B',
 'BAC',
 'RY',
 'HD',
 'DIS',
 'TD',
 'UTX',
 'OXY',
 'AXP',
 'KFT',
 'COP',
 'MO',
 'CAT',
 'V',
 'CMCSA',
 'MMM',
 'USB',
 'AIG',
 'EMC',
 'CVS',
 'BNS',
 'UNH',
 'BA',
 'UPS',
 'BMY',
 'AMGN',
 'UNP',
 'GS',
 'MA',
 'EBAY',
 'DD',
 'HPQ',
 'LLY',
 'CL',
 'SU',
 'SPG',
 'EPD',
 'HON',
 'LVS',
 'ESRX',
 'SBUX',
 'NKE',
 'ACN',
 'F',
 'MDT',
 'SO',
 'MON',
 'TEVA',
 'DOW',
 'GILD',
 'ABX',
 'IMO',
 'LOW',
 'DHR',
 'CNI',
 'DB',
 'TGT',
 'PCLN',
 'BMO',
 'POT',
 'MET',
 'COST',
 'EMR',
 'BIDU',
 'TXN',
 'GM',
 'TWX',
 'CNQ',
 'FCX',
 'APC',
 'PNC',
 'PX',
 'APA',
 'BP',
 'YUM',
 'DTV',
 'DE',
 'NWSA',
 'QQQ',
 'MS',
 'BCE',
 'ENB',
 'BIIB',
 'TJX',
 'KMB',
 'CELG',
 'BAX',
 'TRP',
 'NOV',
 'HAL',
 'D',
 'GG',
 'CM',
 'MCI',
 'WAG'

Let's get financial data now.

In [12]:
#taken from https://www.alpharithms.com/python-financial-data-491110/

In [1]:
pip install yfinance

Could not fetch URL https://pypi.org/simple/yfinance/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/yfinance/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)'))) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)'))) - skipping
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement yfinance (from versions: none)
ERROR: No matching distribution found for yfinance


In [14]:
import yfinance as yf

ModuleNotFoundError: No module named 'yfinance'

In [1]:
congress.info()

NameError: name 'congress' is not defined

In [ ]:
congress['transaction_date'] = pd.to_datetime(congress['transaction_date'])
congress['disclosure_date'] = pd.to_datetime(congress['disclosure_date'])
congress.info()

In [ ]:
congress['year'] = congress['transaction_date'].map(lambda x: x.year)
congress.year.value_counts()

In [ ]:
# Request historical data for past 11 years
apple = yf.Ticker("AAPL").history(period='11y')
# Show info
print(apple.info())

In [ ]:
apple

In [ ]:
apple['Open']

In [ ]:
#1 year rolling average
plt.figure(figsize=(10,6))
plt.plot(apple['Open'])
plt.plot(apple['Open'].rolling(365).mean());

In [ ]:
len(tick_list1)

In [ ]:
congress.head()

In [ ]:
congress['name'].unique()

In [ ]:
congress[congress['name'] == 'A. Mitchell Mcconnell']

In [ ]:
congress[congress['name'] == 'A. Mitchell Mcconnell'][congress['transaction_date'] == '2022-06-03']

In [ ]:
# Request historical data for past 11 years
WFC = yf.Ticker("WFC").history(period='11y')
# Show info
print(WFC.info())

In [ ]:
WFC.head()

In [ ]:
WFC[WFC.index == '2022-06-03']

In [ ]:
WFC[WFC.index == '2022-07-22']

In [ ]:
price_a = float(WFC[WFC.index == '2022-06-03']['Open'])
price_b = float(WFC[WFC.index == '2022-07-22']['Open'])
price_c = price_b - price_a
price_c

We see that the Senator so far has lost money on the trade. But can we atomize this?

In [ ]:
def value_of_congress(congress_person):
    #set-up
    up_to_date = '2022-07-22'
    congress_df = congress[congress['name'] == congress_person][congress['type'] == 'purchase']
    price_list = []
    for i in congress_df['ticker'].unique():
        # Request historical data for past 11 years
        ticker_df = yf.Ticker(i).history(period='11y') 
        #print(i)
        if len(ticker_df) > 0:
            #Creating current values
            up_to_date = str(yf.Ticker(i).history(period='11y')[::-1].index[0])[0:10]
            ticker_end_price = float(ticker_df[ticker_df.index == up_to_date]['Open'])
            for j in congress_df[congress_df['ticker'] == i]['transaction_date']:
                #print(j)
                c = ticker_df[ticker_df.index == str(j)[0:10]]['Open']
                if len(congress_df[congress_df['transaction_date'] == j]['amount']) == 1:
                    val = float(congress_df[congress_df['transaction_date'] == j]['amount'])
                    if str(c)[0] == 'D':
                        #Value of stocks from purchase to current
                        c = (ticker_end_price - float(c)) * (val/float(c))
                        price_list.append(c)
                else:
                    for k in congress_df[congress_df['transaction_date'] == j]['amount']:
                        #print(k)
                        val = float(k)
                        if str(c)[0] == 'D':
                          #Value of stocks from purchase to current
                            c = (ticker_end_price - float(c)) * (val/float(c))
                            price_list.append(c)
    end_value = 0
    for i in price_list:
        #adding together
        end_value += i
    return end_value

In [ ]:
str(yf.Ticker('NUAN').history(period='11y')[::-1].index[0])[0:10]

In [ ]:
value_of_congress('A. Mitchell Mcconnell')

Mitch Mcconnell is losing money. Let's run a loop to create a dictionary, where we can look at other members of congress.

In [ ]:
congress['name'].unique()

In [ ]:
end_price_dict = {}
for i in congress['name'].unique():
    print(i)
    print(value_of_congress(i))
    end_price_dict.update({i:value_of_congress(i)})
end_price_dict

In [ ]:
#Recreating the dictionary by hand
end_price_dict = {
 'A. Mitchell Mcconnell': 6998.211348891493,
 'Abigail Spanberger': 1495.8014028765494,
 'Adam B. Schiff': 0,
 'Adam Kinzinger': 163105.08430793879,
 'Alan S. Lowenthal': 313417.06899300584,
 'Andrew Garbarino': -25832.567550523414,
 'Angus S King': 116009.72414713286,
 'Anthony E. Gonzalez': -427943.2050591054,
 'Aston Donald McEachin': -28589.793354615,
 'August Lee Pfluger': -19295.195081701837,
 'Austin Scott': -617369.4925797061,
 'Benjamin L Cardin': 0,
 'Bill Flores': 0,
 'Bill Pascrell': 0,
 'Billy Long': -17.948827631109598,
 'Blake Moore': -249752.62735958482,
 'Bob Gibbs': 131973.53290073317,
 'Bradley S. Schneider': 0,
 'Brian Higgins': 32227.96834418799,
 'Brian Mast': -511373.5170136403,
 'Carol Devine Miller': 834895.4142762966,
 'Charles J. "Chuck" Fleischmann': -156867.8171747845,
 'Cheri Bustos': 35088.13809046703,
 'Chris Van Hollen': 0,
 'Christopher A Coons': 21877.050460222803,
 'Christopher L. Jacobs': -152096.507122648,
 'Cindy Axne': 532099.43695727,
 'Cory A Booker': 0,
 'Cynthia M Lummis': -623.6467410362519,
 'Dan Newhouse': 0,
 'Daniel Crenshaw': 105393.32249125607,
 'Daniel Meuser': 0,
 'Daniel S Sullivan': 26940.647602483128,
 'David A Perdue ': 19851603.004734807,
 'David B. McKinley': 1635807.7263160802,
 'David Cheston Rouzer': 4968.876443311557,
 'David E. Price': 92069.43967476729,
 'David Kustoff': -174434.67582357454,
 'David Madison Cawthorn': -689369.4555165938,
 'David P. Joyce': 498180.0374936425,
 'David P. Roe': 189344.3990308879,
 'David Trone': 373614.3537910357,
 'Dean Phillips': 2790648.017746348,
 'Debbie Dingell': 42186.30370394174,
 'Debbie Wasserman Schultz': -8843.934491778386,
 'Deborah K. Ross': -349873.6314936659,
 'Diana Harshbarger': 12460.357027505343,
 'Donald Sternoff Beyer': 11527769.310012609,
 'Donald Sternoff Honorable Beyer': 14879.42853923687,
 'Donna Shalala': 711410.7732370538,
 'Doris O. Matsui': 0,
 'Doug Lamborn': -353422.3929436527,
 'Dwight Evans': 129991.21901654186,
 'Earl Blumenauer': 454242.2623562518,
 'Earl Leroy Carter': 0,
 'Ed Case': -40929.95146140495,
 'Ed Perlmutter': -23003.162642051073,
 'Elaine Luria': -590583.3201939834,
 'Felix Barry Moore': -104762.41694230575,
 'Francis Rooney': 619076.2465854705,
 'Frank Pallone': 0,
 'Fred Upton': 111678.86683874745,
 'Garret Graves': -51180.671316140404,
 'Gary C Peters': 343277.10804894177,
 'George Holding': 0,
 'Gerald E. Connolly': 94.77047056295734,
 'Gilbert Cisneros': 4416282.74033472,
 'Greg Gianforte': 5862705.2247966835,
 'Greg Steube': -27752.490695642067,
 'Gus M. Bilirakis': 0,
 'Hakeem S. Jeffries': -3763.1874919342313,
 'Harley E. Rouda': -1735.5574911496853,
 'Harold Dallas Rogers': -26257.88158088081,
 'Jacklyn S Rosen': 0,
 'James Comer': 124731.51314871846,
 'James E Hon Banks': -212972.03615405687,
 'James E. Banks': -54705.92220348897,
 'James French Hill': 162039.13131896494,
 'James Hagedorn': 0,
 'James M Inhofe': 3049782.8959975666,
 'James M. Costa': 0,
 'James R. Langevin': -3076253.571786127,
 'Jerry Moran': 740084.1047230462,
 'Joe Courtney': 342400.4233689962,
 'John A. Yarmuth': 170367.8870345087,
 'John B. Larson': 0,
 'John Boozman': 0,
 'John Cornyn': 251595.98798750626,
 'John Curtis': 863784.5653957713,
 'John F Reed': 1565179.0052891814,
 'John Garamendi': 0,
 'John Hoeven': 14931960.640482292,
 'John N Kennedy': 4852.833983061411,
 'John R Thune': 0,
 'John Rutherford': -62374.53098982082,
 'John W Hickenlooper': -88923.53529685093,
 'Joseph D. Morelle': 0,
 'Joseph Manchin': 28575.325461139408,
 'Joseph P. Kennedy': 0,
 'Josh Gottheimer': -37635385.71554869,
 'Judy Chu': 0,
 'Justin Amash': 49834.468524107986,
 'K. Michael Conaway': 3503919.2981400248,
 'Katherine M. Clark': 457082.0241417676,
 'Kathy Castor': 135789.33686833942,
 'Kathy Manning': -537812.9316241216,
 'Kelly Loeffler': 8429105.449129278,
 'Kenneth R. Buck': 0,
 'Kenny Marchant': 861911.997733904,
 'Kevin Hern': 3081199.910885258,
 'Kim Dr Schrier': -26168.057300280983,
 'Kim Schrier': 0,
 'Kurt Schrader': 158284.81515539164,
 'Ladda Tammy Duckworth': 0,
 'Larry Bucshon': -22337.295279071855,
 'Linda T. Sanchez': 0,
 'Lloyd Doggett': 165450.97780603802,
 'Lloyd K. Smucker': 0,
 'Lois Frankel': 817545.2666488824,
 'Lori Trahan': 0,
 'Maria Cantwell': 0,
 'Marie Newman': -2291325.1128683626,
 'Marjorie Taylor Greene': -144767.9389458471,
 'Mark Dr Green': 22573083.594901323,
 'Mark Green': 36433674.8084397,
 'Mark R Warner': 0,
 'Matt Gaetz': 0,
 'Michael  B Enzi': 0,
 'Michael C. Burgess': -38493.96458335107,
 'Michael F Bennet': 0,
 'Michael Garcia': 734909.0995374752,
 'Michael John Gallagher': 0,
 'Michael K. Simpson': 88543.44362452425,
 'Michael Patrick Guest': -6992.827593336842,
 'Michael T. McCaul': 11641953.95520175,
 'Michael Waltz': -92301.61931500383,
 'Mike Kelly': 241743.75592559925,
 'Mike Rounds': 0,
 'Mikie Sherrill': 549719.5583314941,
 'Mo Brooks': 170069.0264089427,
 'Mr. Peter Meijer': 39323.802140492844,
 'Mrs. Marjorie Taylor Greene': -153448.37321691518,
 'Nancy Pelosi': -7296907.397314373,
 'Neal Patrick Dunn MD, FACS': 0,
 'Neal Patrick MD, FACS Dunn': 0,
 'Neal Patrick MD, Facs Dunn': 0,
 'Nicholas Van Taylor': 0,
 'None Ashley Hinson Arenholz': -381.7337431242722,
 'None Deborah K. Ross': 45461.83914303286,
 'None Kathy Manning': 56357.55198095854,
 'None Marie Newman': -142410.43982821042,
 'None Sara Jacobs': 7506.019889692586,
 'None Victoria Spartz': 4710.076028453936,
 'Pat Roberts': 6309752.612168883,
 'Patrick Fallon': -2510691.2891932856,
 'Patrick J Toomey': 387337.28558225697,
 'Patrick T. McHenry': 0,
 'Patty Murray': 517457.16057706275,
 'Pete Sessions': -183584.93413129705,
 'Peter J. Visclosky': 0,
 'Peter Welch': 11066.26664309714,
 'Rafael E Cruz': -21344.129299399247,
 'Rand Paul': -795.742592290793,
 'Raúl M. Grijalva': -1635.3802180399257,
 'Richard W. Allen': 183148.82183794578,
 'Robert C. "Bobby" Scott': 0,
 'Robert E. Latta': 14436.820236735706,
 'Robert J. Wittman': 200445.77145875647,
 'Robert P Casey': 0,
 'Roger F Wicker': 3144.712755898532,
 'Roger W. Marshall': 105702.14430195422,
 'Rohit Khanna': 104943.86574775072,
 'Ron Estes': -43233.07664722893,
 'Ron L Wyden': 2389196.5227998905,
 'Roy Blunt': -12023.121502792254,
 'Sara Jacobs': -2143.286113984107,
 'Scott Franklin': -274074.8953662389,
 'Scott H. Peters': 0,
 'Sean Patrick Maloney': 0,
 'Sharice Davids': 0,
 'Sheldon Whitehouse': 7894262.229120037,
 'Shelley M Capito': 1801835.6130269652,
 'Stephanie Bice': 0,
 'Steve Cohen': -44252.98618885268,
 'Susan A. Davis': 296203.3695693297,
 'Susan M Collins': 3249889.4624370434,
 'Susan W. Brooks': 236818.3123773496,
 'Susie Lee': 792669.1081153743,
 'Suzan K. DelBene': -5674795.2507581,
 'Tammy Duckworth': 0,
 'Thomas H Tuberville': -594648.9389402983,
 'Thomas Massie': 0,
 'Thomas R Carper': 1731575.9207783851,
 'Thomas R Tillis': 0,
 'Thomas Suozzi': -418978.0745905322,
 'Thomas Udall': 0,
 'Tim Burchett': 0,
 'Timothy M Kaine': 0,
 'Tina Smith': 0,
 'Tom Cole': -4848.57974438228,
 'Tom Malinowski': -942102.5388914375,
 "Tom O'Halleran": 0,
 'Tom Rice': 152746.5966929144,
 'Trey Hollingsworth': 2032911.4652916088,
 'Vicente Gonzalez': 0,
 'Victoria Spartz': -573500.4822680631,
 'Virginia Foxx': 990658.1226034971,
 'W. Greg Steube': 6052.7611287816235,
 'William Cassidy': 819535.3436767141,
 'William F Hagerty': -41450.183223478205,
 'William R. Keating': 2139.3653559638915,
 'William R. Timmons': 6270.425943133285,
 'Wm. Lacy Clay': 0,
 'Zoe Lofgren': -210277.9431548952
}

In [ ]:
millions_list = [n for n in end_price_dict if end_price_dict[n] >= 1000000]
millions_list

These are the Senators and Representatives that we suspect to have made at least a million dollars. Let's incorporate the values and see if we can find any trends.

In [ ]:
congress.columns

In [ ]:
congress['values'] = congress['name'].map(lambda x: end_price_dict[x])
congress.head()

In [ ]:
congress = congress.drop(columns="Unnamed: 0")
congress.head()

In [ ]:
sns.boxplot(data=congress, x='party', y='values')

This shows that the Republicans tend to make more in stock trades than the Democrats

In [ ]:
sns.boxplot(data=congress, x='gender', y='values')

Males and females are about the same here

In [ ]:
sns.boxplot(data=congress, x='chamber', y='values')

Senators seem to make more than the House

In [ ]:
congress.head()

Let's make the columns more usable. 

In [ ]:
congress['senate'] = congress['chamber'].map(lambda x: 1 if x == 'senate' else 0)
congress['republican'] = congress['party'].map(lambda x: 1 if x == 'Republican' else 0)
congress['male'] = congress['gender'].map(lambda x: 1 if x == 'M' else 0)
congress['age'] = congress['birthday'].map(lambda x: 2022 - int(x[0:4]))

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(congress.corr()[['values']].sort_values(by='values',ascending=False),
            annot = True,
            vmin = -1,
            vmax = 1,
            cmap = 'coolwarm');

In [ ]:
congress['state'] = congress['represents'].map(lambda x: x[0:2])

In [ ]:
plt.figure(figsize=(30,20))
sns.scatterplot(
    x = 'birthday',
    y = 'values',
    hue = 'state',
    data = congress,
    palette = 'bright'
)